In [81]:
import cProfile
import os
import pstats
import time

import psutil
import torch
import torch.nn as nn
import torch.optim as optim

from panther.nn.attention import RandMultiHeadAttention
from panther.nn.pawXimpl import sinSRPE

In [82]:
def profile(func, *args):
    with cProfile.Profile() as pr:
        rets = func(*args)
    results = pstats.Stats(pr).sort_stats(pstats.SortKey.TIME)
    results.print_stats()
    return rets

In [83]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


# Sample synthetic dataset (sequence classification)
def generate_data(num_samples=100, seq_len=3000, dim=32, num_classes=2):
    X = torch.randn(num_samples, seq_len, dim, device=device)  # Random embeddings
    y = torch.randint(0, num_classes, (num_samples,), device=device)  # Random labels
    return X, y


def get_cpu_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss  # in bytes


# Define a simple Transformer-like model with custom Multihead Attention
class CustomTransformer(nn.Module):
    def __init__(self, embed_dim, num_heads, num_classes, num_rand_features=None):
        super().__init__()
        if num_rand_features is None:
            self.attention = nn.MultiheadAttention(
                embed_dim, num_heads, batch_first=True
            )
        else:
            spre = sinSRPE(
                num_heads, embed_dim // num_heads, 16, 32, device, torch.float32
            )
            spre = None
            self.attention = RandMultiHeadAttention(
                embed_dim,
                num_heads,
                num_random_features=num_rand_features,
                kernel_fn="softmax",
                SRPE=spre,
            )
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.attention(x, x.clone(), x.clone())[0]  # Multihead Attention
        x = x.mean(dim=1)  # Pooling
        return self.fc(x)


def train_model(model, X_train, y_train, epochs=5, lr=1e-3):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.train()
    start_time = time.time()

    if device.type == "cuda":
        torch.cuda.reset_peak_memory_stats(device)
    else:
        mem_before = get_cpu_memory_usage()

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    training_time = time.time() - start_time

    if device.type == "cuda":
        mem_usage = torch.cuda.max_memory_allocated(device)
    else:
        mem_usage = get_cpu_memory_usage() - mem_before

    return loss.item(), training_time, mem_usage


# Forward pass benchmarking
def measure_forward_pass(model, X):
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        if device.type == "cuda":
            torch.cuda.reset_peak_memory_stats(device)
            torch.cuda.synchronize()
        else:
            mem_before = get_cpu_memory_usage()

        start_time = time.time()
        _ = model(X)
        if device.type == "cuda":
            torch.cuda.synchronize()
        forward_time = time.time() - start_time

        if device.type == "cuda":
            mem_usage = torch.cuda.max_memory_allocated(device)
        else:
            mem_usage = get_cpu_memory_usage() - mem_before

    return forward_time, mem_usage


def measure_accuracy(model, X, y_true):
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(X)

        if outputs.shape[-1] == 1:  # Binary classification
            preds = (outputs > 0.5).long().squeeze()
        else:  # Multiclass classification
            preds = outputs.argmax(dim=1)

        accuracy = (preds == y_true).float().mean().item()

    return accuracy


# Hyperparameters
embed_dim = 32
num_heads = 4
num_classes = 2


# Generate synthetic dataset
def fun(s):
    X_train, y_train = generate_data(seq_len=s)

    # Initialize models
    custom_attention_model = CustomTransformer(
        embed_dim, num_heads, num_classes, num_rand_features=128
    )
    torch_attention_model = CustomTransformer(embed_dim, num_heads, num_classes)
    # Compare models
    # torch_loss, torch_time, torch_mem = profile(
    #     train_model, torch_attention_model, X_train, y_train
    # )
    # print("---------------------------------------------------")
    # custom_loss, custom_time, custom_mem = profile(
    #     train_model, custom_attention_model, X_train, y_train
    # )
    # print("---------------------------------------------------")
    torch_loss, torch_time, torch_mem = train_model(
        torch_attention_model, X_train, y_train
    )
    custom_loss, custom_time, custom_mem = train_model(
        custom_attention_model, X_train, y_train
    )
    custom_forward_time, custom_forward_mem = measure_forward_pass(
        custom_attention_model, X_train
    )
    torch_forward_time, torch_forward_mem = measure_forward_pass(
        torch_attention_model, X_train
    )
    # Print results
    print(
        f"Custom Attention - Loss: {custom_loss:.4f}, Time: {custom_time:.4f}s, Memory: {custom_mem / 1024:.2f} KB,Memory per length: {custom_mem / 1024 / s:.2f} KB"
    )
    print(
        f"Torch Attention - Loss: {torch_loss:.4f}, Time: {torch_time:.4f}s, Memory: {torch_mem / 1024:.2f} KB,Memory per length: {torch_mem / 1024 / s:.2f} KB"
    )
    print(
        f"Custom Attention - Forward Time: {custom_forward_time:.4f}s, Forward Memory: {custom_forward_mem / 1024:.2f} KB,Memory per length: {custom_forward_mem / 1024 / s:.2f} KB"
    )
    print(
        f"Torch Attention - Forward Time: {torch_forward_time:.4f}s, Forward Memory: {torch_forward_mem / 1024:.2f} KB,Memory per length: {torch_forward_mem / 1024 / s:.2f} KB"
    )


In [84]:
from panther.nn.attention import make_performer


def fun2(s):
    results = []

    X_train, y_train = generate_data(seq_len=s)
    torch_attention_model = CustomTransformer(embed_dim, num_heads, num_classes)
    torch_loss, torch_time, torch_mem = train_model(
        torch_attention_model, X_train, y_train
    )
    torch_acc = measure_accuracy(torch_attention_model, X_train, y_train)
    print(f"[s={s}] Torch Attention - Accuracy: {torch_acc:.4f}")

    for j in [64, 128, 256, 512, 1024]:
        print(f"Num Random Features: {j}")
        custom_attention_model = CustomTransformer(
            embed_dim, num_heads, num_classes, num_rand_features=j
        )
        custom_attention_model.fc = torch_attention_model.fc
        custom_attention_model.attention = make_performer(
            torch_attention_model.attention,
            num_random_features=j,
            kernel_fn="softmax",
            iscausal=False,
        )
        custom_acc = measure_accuracy(custom_attention_model, X_train, y_train)
        print(f"Custom Attention - Accuracy: {custom_acc:.4f}")
        results.append((j, custom_acc))

    return torch_acc, results  # base model accuracy and list of (num_features, acc)


def fun3(s):
    X_train, y_train = generate_data(seq_len=s)
    custom_attention_model = CustomTransformer(
        embed_dim, num_heads, num_classes, num_rand_features=128
    )
    c_loss, c_time, c_mem = train_model(
        custom_attention_model, X_train, y_train, epochs=10
    )
    print(
        f"Custom Attention - Loss: {c_loss:.4f}, Time: {c_time:.4f}s, Memory: {c_mem / 1024:.2f} KB, Memory per length: {c_mem / 1024 / s:.2f} KB"
    )
    acc = measure_accuracy(custom_attention_model, X_train, y_train)
    print(f"[s={s}] Torch Attention - Accuracy: {acc:.4f}")


fun3(3000)


Custom Attention - Loss: 0.6851, Time: 0.2305s, Memory: 5570183.00 KB, Memory per length: 1856.73 KB
[s=3000] Torch Attention - Accuracy: 0.5500


In [85]:
sequence_lengths = [10, 100, 1000]
all_results = {}

# for s in sequence_lengths:
#     torch_acc, custom_results = fun2(s)
#     all_results[s] = {
#         "torch": torch_acc,
#         "custom": custom_results,
#     }

# # Plotting
# for s in sequence_lengths:
#     custom_results = all_results[s]["custom"]
#     torch_acc = all_results[s]["torch"]

#     j_vals = [j for j, _ in custom_results]
#     acc_vals = [acc for _, acc in custom_results]

#     plt.figure()
#     plt.plot(j_vals, acc_vals, marker="o", label="Custom Attention")
#     plt.axhline(y=torch_acc, color="r", linestyle="--", label="Torch Attention")
#     plt.title(f"Accuracy vs Random Features (seq_len={s})")
#     plt.xlabel("Number of Random Features")
#     plt.ylabel("Accuracy")
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()


In [86]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RandMultiHeadAttention(
    embed_dim=8,
    num_heads=2,
    num_random_features=128,
    dropout=0.1,
    kernel_fn="softmax",
    iscausal=False,
    device=device,
    dtype=torch.float32,
).to(device)

model(
    query=torch.randn(1, 2, 8).to(device),
    key=torch.randn(1, 3, 8).to(device),
    value=torch.randn(1, 3, 8).to(device),
    attention_mask=torch.tensor([[[1, 1, 1], [0, 0, 0]]], dtype=torch.bool).to(device),
)

(tensor([[[ 0.4562,  0.0532, -0.3268, -0.1668, -0.2412,  0.5843, -0.5799,
            0.4447],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000]]], device='cuda:0', grad_fn=<ViewBackward0>),
 None)

In [87]:
query = torch.randn(1, 2, 2, 4)
key = torch.randn(1, 3, 2, 4)
value = torch.randn(1, 3, 2, 4)
attention_mask = torch.tensor([[[1, 1, 1], [0, 0, 0]]], dtype=torch.bool)

In [88]:
maskL = attention_mask.all(-1)
maskS = attention_mask.all(-2)
maskL.shape, maskS.shape

(torch.Size([1, 2]), torch.Size([1, 3]))

In [89]:
maskL = maskL.unsqueeze(-1).unsqueeze(-1)
maskS = maskS.unsqueeze(-1).unsqueeze(-1)
maskL.shape, maskS.shape

(torch.Size([1, 2, 1, 1]), torch.Size([1, 3, 1, 1]))

In [90]:
query * maskL

tensor([[[[-2.5095,  0.4880,  0.7846,  0.0286],
          [ 0.6408,  0.5832,  1.0669, -0.4502]],

         [[-0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000]]]])